In [4]:
# for tensorboard log folder name
from time import time

# working with, mainly resizing, images
import cv2                 

# dealing with arrays
import numpy as np         

# dealing with directories
import os                  

# mixing up or currently ordered data that might lead our network astray in training.
from random import shuffle 

# a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
from tqdm import tqdm      

# neural networks
#import tflearn
#from tflearn.layers.conv import conv_2d, max_pool_2d
#from tflearn.layers.core import input_data, dropout, fully_connected
#from tflearn.layers.estimator import regression
import tensorflow as tf
#from keras.callbacks import TensorBoard

# use PlaidML as backend for hardware acceleration
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


TRAIN_DIR = './train'
TEST_DIR = './test'
IMG_SIZE = 64
#LR = 1e-3
BATCH_SIZE = 32

# just so we remember which saved model is which, sizes must match
MODEL_NAME = './dogs-vs-cats-{}.model'.format('2conv-basic1') 

def label_img(imgname):
    file_extension = imgname.split('.')[-1]
    if file_extension != 'jpg':
        return []
    word_label = imgname.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

    
def create_train_data():
    training_data = []
    img = []
    for imgname in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(imgname)
        if label == []:
            continue
        try:
            path = os.path.join(TRAIN_DIR,imgname)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(img),np.array(label)])
        except Exception as e:
            print("error with image: '{}', exception: {}".format(imgname, str(e)))
    shuffle(training_data)
    np.save('./train_data_64.npy', training_data)
    return training_data


def create_test_data():
    testing_data = []
    for imgname in tqdm(os.listdir(TEST_DIR)):
        try:
            path = os.path.join(TEST_DIR,imgname)
            img_num = imgname.split('.')[0]
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            testing_data.append([np.array(img), img_num])
        except Exception as e:
            print("error with image: '{}', exception: {}".format(imgname, str(e)))
    shuffle(testing_data)
    np.save('./test_data_64.npy', testing_data)
    return testing_data


# Check if training data file alrady exists
train_data_exists = os.path.isfile('./train_data_64.npy')
if train_data_exists:
    # We have already created the dataset
    train_data = np.load('./train_data_64.npy')
else:
    # Create the dataset
    train_data = create_train_data()

# Check if testing data file alrady exists
test_data_exists = os.path.isfile('./test_data_64.npy')
if test_data_exists:
    # We have already created the dataset
    test_data = np.load('./test_data_64.npy')
else:
    # Create the dataset
    test_data = create_test_data()

    
    
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(64, 64, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(2,  activation=tf.nn.softmax)
])
    
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

if os.path.exists('./{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

    
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/{}".format(time()))

#model.fit(
#    {'input': X}, 
#    {'targets': Y}, 
#    n_epoch=1, 
#    validation_set=(
#        {'input': test_x}, 
#        {'targets': test_y}), 
#    snapshot_step=500, 
#    show_metric=True, 
#    run_id=MODEL_NAME,
#    callbacks=[tensorboard])

model.fit(X, 
          Y, 
          epochs=1, 
          steps_per_epoch=500,
          validation_split=0.1,
          #validation_data=(test_x, test_y),
          callbacks=[tensorboard])


model.save(MODEL_NAME)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 24446 arrays: [array([[1],
       [0]]), array([[1],
       [0]]), array([[1],
       [0]]), array([[1],
       [0]]), array([[0],
       [1]]), array([[1],
       [0]]), array([[0],
       [1]]), array([[0],
     ...